# fiberassign on Mock Catalogs

**NOTE**: this tutorial takes ~30 minutes just to run all the steps

In [ ]:
import os, sys, subprocess, fitsio, shutil
from pkg_resources import resource_filename
from collections import Counter

import numpy as np
from astropy.table import Table
from astropy.io import fits

import desimodel.io
import desimodel.focalplane
import desimodel.footprint
from desitarget.targetmask import desi_mask, obsconditions
import matplotlib.pyplot as plt

## Learning goals

In this notebook you will learn how to:

* Generate mock files from simulations to feed into fiberassign.
* Run fiberassign using the files generated above.
* Explore the outputs of fiberassign to estimate its efficiency.


See https://desi.lbl.gov/trac/wiki/Computing/JupyterAtNERSC for instructions on configuring jupyter kernels with pre-installed DESI software at NERSC.  This tutorial was last tested with the 22.2 kernel on April 2022.

This notebook reuses code in the [main fiberassign notebook](https://github.com/desihub/tutorials/blob/master/FiberAssign.ipynb).

In [ ]:
workdir = os.path.join(os.environ['SCRATCH'], 'desi', 'test', 'fiberassign_mock')
os.makedirs(workdir, exist_ok=True)
outdir = os.path.join(workdir, 'output')
os.makedirs(outdir, exist_ok=True)
shutil.copy(resource_filename('desitarget.mock', 'data/select-mock-targets.yaml'), workdir)
os.chdir(workdir)

In [ ]:
# This is the content of the configuration file to be passed to select_mock_targets
!cat select-mock-targets.yaml

Some more files needed to run fiberassign

In [ ]:
mtlfile = os.path.join(workdir, 'mtl.fits')
truthfile = os.path.join(workdir, 'truth.fits')
starfile = os.path.join(workdir, 'std.fits')
targetcutfile = os.path.join(workdir, 'targets.fits') 
skycutfile = os.path.join(workdir, 'sky.fits') 
tilefile = os.path.join(workdir, "tiles.fits")

In [ ]:
# tile selection
program = "dark"

tiles = desimodel.io.load_tiles()
bright = tiles['PROGRAM']=='BRIGHT'
    
small = ((tiles['RA']>12) & (tiles['RA']<20) & (tiles['DEC']<1) & (tiles['DEC']>-1))

if program=="bright":
    Table(tiles[(bright)&(small)]).write(tilefile, overwrite=True)
else:
    Table(tiles[(~bright) & (small)]).write(tilefile, overwrite=True)

print("Wrote tiles to {}".format(tilefile))

In [ ]:
!pwd
!ls

In [ ]:
cmd = "srun -A desi -N 2 -n 16 -c 8"  # -c 8 not 4 for hyperthreading
cmd += " -C haswell -t 01:00:00 --qos interactive "
cmd += " mpi_select_mock_targets --no-spectra --nproc 4 --nside 32 --seed 10"
cmd += " -c ./select-mock-targets.yaml "
cmd += " --output_dir ./ "
cmd += " --tiles ./tiles.fits"
print(cmd)

In [ ]:
# To run target selection comment out these lines. it should take ~30 minutes
print('RUNNING: '+cmd)
try:
    results = subprocess.check_output(cmd.split(), stderr=subprocess.STDOUT)
    print(results.decode())
    print('--- SUCCESS ---')
except subprocess.CalledProcessError as ex:
    print('--- ERROR {} ---'.format(ex.returncode))
    print(ex.output.decode())
    raise ex

In [ ]:
cmd = 'join_mock_targets --overwrite --mockdir ./'
print('RUNNING: '+cmd)
try:
    results = subprocess.check_output(cmd.split(), stderr=subprocess.STDOUT)
    print(results.decode())
    print('--- SUCCESS ---')
except subprocess.CalledProcessError as ex:
    print('--- ERROR {} ---'.format(ex.returncode))
    print(ex.output.decode())
    raise ex

In [ ]:
mtldata = Table.read('mtl-dark.fits')
plt.scatter(mtldata['RA'], mtldata['DEC'],s=0.1, alpha=0.1)
plt.xlabel('RA')
plt.ylabel('DEC')

By default, fiberassign uses the real focalplane geometry and properties at the current time.  During operations, if a fiber broke yesterday and you run fiberassign today with an up-to-date desimodel data checkout, then that fiber will not be assigned.  For this tutorial, we will run fiberassign with a *rundate* set to a time in the past before the start of commissioning.  This will give us a nominal focalplane layout with all positioners working.

In [ ]:
assign_date = "2022-01-01T00:00:00+00:00"

In [ ]:
# Run fiberassign
cmd = 'fiberassign --overwrite --mtl mtl-dark.fits --sky sky.fits'
cmd += ' --rundate {}'.format(assign_date)
cmd += ' --footprint ./tiles.fits'
cmd += ' --outdir ./output/'
cmd = cmd.format(outdir=outdir)

In [ ]:
print('RUNNING: '+cmd)
try:
    results = subprocess.check_output(cmd.split(), stderr=subprocess.STDOUT)
    print(results.decode())
    print('--- SUCCESS ---')
except subprocess.CalledProcessError as ex:
    print('--- ERROR {} ---'.format(ex.returncode))
    print(ex.output.decode())
    raise ex

In [ ]:
!ls output

In [ ]:
tiles = Table.read("tiles.fits")
targets = Table.read("mtl-dark.fits")

In [ ]:
# Gather all outputs
assignments = list()
for tileid in tiles['TILEID']:
    fafile = '{}/fiberassign-{:06d}.fits'.format(outdir, tileid)
    assignments.append(Table.read(fafile, hdu=1))
    
assigned_targetids = np.concatenate([tmp['TARGETID'] for tmp in assignments])
isAssigned = np.in1d(targets['TARGETID'], assigned_targetids)

plt.figure(figsize=(12,4))
plt.plot(targets['RA'][isAssigned], targets['DEC'][isAssigned], 'k,')
plt.title('Targets assigned to fibers')
plt.xlabel('RA [deg]')
plt.ylabel('DEC [deg]')

In [ ]:
plt.figure(figsize=(12,4))
plt.plot(targets['RA'][~isAssigned], targets['DEC'][~isAssigned], 'k,')
plt.title('Targets left unassigned to fibers')
plt.xlabel('RA [deg]')
plt.ylabel('DEC [deg]')